In [2]:
import pandas as pd

### Getting the DF

In [14]:
df = pd.read_csv('data/twitter.csv')

In [15]:
df.head()

,Unnamed: 0,canton_code,canton,month,language,gender,time_period,count,sentiment
0,0,AG,Aargau,8,English,Male,Day,50,0.16
1,1,AG,Aargau,10,English,Female,Day,10,0.10
2,2,AG,Aargau,2,German,Unknown,Day,6,1.00
3,3,AG,Aargau,3,German,Female,Day,6,1.00
4,4,AG,Aargau,4,German,Unknown,Night,5,1.00


In [16]:
df.canton.unique()

array(['Aargau', 'Basel-City', 'Bern', 'Fribourg', 'Geneva', 'Glarus',
       'Grisons', 'Jura', 'Lucerne', 'Neuchâtel', 'Nidwalden', 'Obwalden',
       'Schaffhausen', 'Schwyz', 'Solothurn', 'Saint Gallen', 'Thurgau',
       'Ticino', 'Uri', 'Valais', 'Vaud', 'Zurich'], dtype=object)

In [17]:
df = df.replace('Neuchâtel', 'Neuchatel')

In [21]:
df.to_csv('data/twitter.csv')

In [20]:
len(df.canton.unique())

22

In [8]:
len(df.canton.unique())

22

In [9]:
code_mapping = pd.read_csv('data/canton_code.csv').ix[:,1:3]

In [10]:
code_mapping.columns = ['canton_code', 'canton']

In [11]:
df.head()

,Unnamed: 0,canton_code_x,canton,canton_code_y,month,language,gender,time_period,count,sentiment
0,0,AG,Aargau,AG,8,NaN,NaN,Day,50,0.16
1,1,AG,Aargau,AG,10,NaN,NaN,Day,10,0.10
2,2,AG,Aargau,AG,2,NaN,NaN,Day,6,1.00
3,3,AG,Aargau,AG,3,NaN,NaN,Day,6,1.00
4,4,AG,Aargau,AG,4,NaN,NaN,Night,5,1.00


In [12]:
new_df = df.copy()

In [13]:
new_df.head()

,Unnamed: 0,canton_code_x,canton,canton_code_y,month,language,gender,time_period,count,sentiment
0,0,AG,Aargau,AG,8,NaN,NaN,Day,50,0.16
1,1,AG,Aargau,AG,10,NaN,NaN,Day,10,0.10
2,2,AG,Aargau,AG,2,NaN,NaN,Day,6,1.00
3,3,AG,Aargau,AG,3,NaN,NaN,Day,6,1.00
4,4,AG,Aargau,AG,4,NaN,NaN,Night,5,1.00


In [ ]:
code_mapping.canton.unique()

In [ ]:
new_df = code_mapping.merge(new_df, on = 'canton')

In [ ]:
new_df.drop(["Unnamed: 0"],1, inplace=True)
# changes avg_sentiment to sentiment
# new_df['sentiment'] = new_df['avg_sentiment']
# new_df = new_df.drop('avg_sentiment',axis=1)

In [ ]:
len(new_df.canton.unique())

In [ ]:
new_df['gender'] = new_df['gender'].map({"FEMALE":"Female","MALE":"Male", "UNKNOWN":"Unknown"})
new_df['language'] = new_df['language'].map({"de":"German","en":"English", "fr":"French"})

In [ ]:
new_df.to_csv('data/twitter.csv')

In [ ]:
new_df.head()

### Creating the Function

In [ ]:
# function that filters out the dataframe and gives the final result wanted.
# gender, month, language

def filter_df(df, gender=True, min_month=1, max_month=10, language = True, time_period = True):
    if type(gender) is str:
        gender = (df['gender'] == gender)
    month = ((min_month <= df['month']) & (df['month'] <= max_month))
    if type(language) is str:
        language = (df['language'] == language)
    if type(time_period) is str:
        time_period = (df['time_period'] == time_period)
    filtered = df[gender & time_period & language]
    return filtered

In [ ]:
cantons_kml = ["ZH","BE","LU","UR","SZ","OW","NW","GL","ZG","FR","SO","BS","BL","SH","AR","AI","SG","GR","AG","TG","TI","VD","VS","NE","GE","JU"]
cantons_df = pd.DataFrame(cantons_kml)
cantons_df.columns = ['canton_code']


def group(filtered_df):
    filtered_df = filtered_df.groupby('canton_code').mean()
    filtered_df = filtered_df.reset_index()[['canton_code','sentiment']] # group by
    final = cantons_df.merge(filtered_df, on='canton_code', how='left')
    return final

In [ ]:
cantons_df.to_csv('cantons_kml.csv')

In [ ]:
def new_df(df, gender=True, min_month=1, max_month=10, language = True, time_period = True):
    return group(filter_df(df,gender,min_month,max_month,language,time_period))

In [ ]:
new_df(df, 'FEMALE',language='en')